In [28]:
import urllib.request
import json
import spotipy
import webbrowser
import pandas
import spotipy.util as util

In [29]:
%run Weather.ipynb
%run OuraRing.ipynb
%run News.ipynb

{'latitude': 51.5, 'longitude': -0.120000124, 'generationtime_ms': 0.032067298889160156, 'utc_offset_seconds': 0, 'timezone': 'Europe/London', 'timezone_abbreviation': 'GMT', 'elevation': 23.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'weather_code': 'wmo code'}, 'current': {'time': '2023-11-29T22:30', 'interval': 900, 'weather_code': 2}}
1
Stored 'weather_mood' (int)


C:\Users\Jessie Liang\Creative Code\ChatgptSpotify\venv\Lib\site-packages\IPython\extensions\storemagic.py:229: UserWarning: using autorestore/weather_mood requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


{'data': [{'id': '48f376dc-9a40-470e-bb2f-644b19de43e6', 'contributors': {'activity_balance': 83, 'body_temperature': 100, 'hrv_balance': None, 'previous_day_activity': 76, 'previous_night': 24, 'recovery_index': 53, 'resting_heart_rate': 89, 'sleep_balance': 52}, 'day': '2023-11-29', 'score': 64, 'temperature_deviation': -0.07, 'temperature_trend_deviation': -0.04, 'timestamp': '2023-11-29T00:00:00+00:00'}], 'next_token': None}


In [30]:
# user inputs their start station
station_start_input = input('Enter')

# remove spaces from user input
input_start_ns = station_start_input.replace(" ", "")

# get station id from user input
try:
    url = (f"https://api.tfl.gov.uk/StopPoint/Search?query={input_start_ns}")

    hdr ={
    # Request headers
    'Cache-Control': 'no-cache',
    }

    req = urllib.request.Request(url, headers=hdr)

    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)
    station_start = response.read()
except Exception as e:
    print(e)

Enter Canada Water Underground Station


In [31]:
station_start_decoded = station_start.decode('utf-8')
station_start_json = json.loads(station_start_decoded)
station_start_json


{'$type': 'Tfl.Api.Presentation.Entities.SearchResponse, Tfl.Api.Presentation.Entities',
 'query': 'CanadaWaterUndergroundStation',
 'total': 1,
 'matches': [{'$type': 'Tfl.Api.Presentation.Entities.MatchedStop, Tfl.Api.Presentation.Entities',
   'parentId': 'HUBZCW',
   'icsId': '1000037',
   'topMostParentId': 'HUBZCW',
   'modes': ['tube'],
   'zone': '2',
   'id': '940GZZLUCWR',
   'name': 'Canada Water Underground Station',
   'lat': 51.497931,
   'lon': -0.049405}]}

In [32]:
# station start id 
id_start = station_start_json['matches'][0]['id']

In [42]:
# user inputs their end station

station_end_input = input('Enter')

# remove spaces from user input

input_end_ns = station_end_input.replace(" ", "")

# get station id from user input

try:
    url = (f"https://api.tfl.gov.uk/StopPoint/Search?query={input_end_ns}")

    hdr ={
    # Request headers
    'Cache-Control': 'no-cache',
    }

    req = urllib.request.Request(url, headers=hdr)

    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)
    station_end = response.read()
except Exception as e:
    print(e)

Enter Farringdon Underground Station


In [43]:
# station end id
station_end_decoded = station_end.decode('utf-8')
station_end_json = json.loads(station_end_decoded)
station_end_json
id_end = station_end_json['matches'][0]['id']
id_end

'940GZZLUFCN'

In [44]:
# use station start id to find crowd level (percentage of baseline)
try:
    url = (f"https://api.tfl.gov.uk/crowding/{id_start}/Live")

    hdr ={
    # Request headers
    'Cache-Control': 'no-cache',
    }

    req = urllib.request.Request(url, headers=hdr)

    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)
    station_start_data = response.read()
except Exception as e:
    print(e)

station_start_data

b'{"dataAvailable":true,"percentageOfBaseline":0.35966676,"timeUtc":"2023-11-29T22:21:00.000Z","timeLocal":"2023-11-29 22:21:00"}'

In [45]:
station_start_data_decoded = station_start_data.decode('utf-8')
station_start_data_decoded_json = json.loads(station_start_data_decoded)

# Access and print the "percentageOfBaseline" value
percentage_of_baseline_start = station_start_data_decoded_json["percentageOfBaseline"]
print(percentage_of_baseline_start)

tube_crowd_start = percentage_of_baseline_start

0.35966676


In [46]:
# use station end id to find crowd level (percentage of baseline)


try:
    url = (f"https://api.tfl.gov.uk/crowding/{id_end}/Live")

    hdr ={
    # Request headers
    'Cache-Control': 'no-cache',
    }

    req = urllib.request.Request(url, headers=hdr)

    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)
    station_end_data = response.read()
except Exception as e:
    print(e)

station_end_data_decoded = station_end_data.decode('utf-8')
station_end_data_decoded_json = json.loads(station_end_data_decoded)

# Access and print the "percentageOfBaseline" value
percentage_of_baseline_end = station_end_data_decoded_json["percentageOfBaseline"]
print(percentage_of_baseline_end)

tube_crowd_end = percentage_of_baseline_end

0.37759057


In [47]:
# get average of station start and end crowd level 

average_crowd = ((tube_crowd_start + tube_crowd_end)/2) 
average_crowd

0.368628665

In [48]:
# round to two decimals (optional)

cs = round(average_crowd,2)

In [49]:
# create variable for bpm input in spotify to get tracks of relevant tempo

tfl_min = 0
tfl_max = 0
if cs < 0.2:
    tfl_min = 0
    tfl_max = 50
elif cs > 0.2 and cs < 0.4:
    tfl_min = 50
    tfl_max = 100
elif cs > 0.4 and cs < 0.6:
    tfl_min = 100
    tfl_max = 150
elif cs > 0.6 and cs < 0.8:
    tfl_min = 150
    tfl_max = 200
elif cs > 0.8:
    tfl_min = 200
    tfl_max = 250

# tfl_min and tfl_max go into min/max tempos in spotify playlist generater code 

print(tfl_min)
print(tfl_max)
    

50
100


In [50]:
credentials = "spotify_keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [51]:
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]

In [52]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-read-recently-played'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [53]:
sp = spotipy.Spotify(auth=token)

In [54]:
recently_played = sp.current_user_recently_played(limit=1,after=None, before=None)
recently_played

{'items': [{'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5XenQ7XfcvQdfIbpLEFaKQ'},
       'href': 'https://api.spotify.com/v1/artists/5XenQ7XfcvQdfIbpLEFaKQ',
       'id': '5XenQ7XfcvQdfIbpLEFaKQ',
       'name': 'GoldLink',
       'type': 'artist',
       'uri': 'spotify:artist:5XenQ7XfcvQdfIbpLEFaKQ'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'DE',
      'EC',
      'EE',
      'SV',
      'FI',
      'FR',
      'GR',
      'GT',
      'HN',
      'HK',
      'HU',
      'IS',
      'IE',
      'IT',
      'LV',
      'LT',
      'LU',
      'MY',
      'MT',
      'MX',
      'NL',
      'NZ',
      'NI',
      'NO',
      'PA',
      'PY',
      'PE',
      'PH',
      'PL',
      'PT',
      'SG',
      'SK',
      'ES',
      'SE',
      

In [55]:
song = recently_played['items']
song

[{'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5XenQ7XfcvQdfIbpLEFaKQ'},
      'href': 'https://api.spotify.com/v1/artists/5XenQ7XfcvQdfIbpLEFaKQ',
      'id': '5XenQ7XfcvQdfIbpLEFaKQ',
      'name': 'GoldLink',
      'type': 'artist',
      'uri': 'spotify:artist:5XenQ7XfcvQdfIbpLEFaKQ'}],
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
     'IE',
     'IT',
     'LV',
     'LT',
     'LU',
     'MY',
     'MT',
     'MX',
     'NL',
     'NZ',
     'NI',
     'NO',
     'PA',
     'PY',
     'PE',
     'PH',
     'PL',
     'PT',
     'SG',
     'SK',
     'ES',
     'SE',
     'CH',
     'TW',
     'TR',
     'UY',
     'US',
     'GB',
     

In [56]:
artist_id = [song[0]['track']['artists'][0]['id']]
artist_id

['5XenQ7XfcvQdfIbpLEFaKQ']

In [57]:
track_id = [song[0]['track']['id']]
track_id

['564oa00vY05d1uYnTEAAmE']

In [58]:
 tracks = sp.recommendations(artist_id,track_id, target_danceability=fnews_score, target_energy=oura, target_valence=weather_mood, min_tempo=tfl_min, max_tempo= tfl_max, genres=None, limit=15)
track_list = tracks['tracks']
uri_list = []
for track in track_list:
    uri_list.append(track['uri'])

uri_list


['spotify:track:1IFIZTpvfeQ3CRrH37ydOR',
 'spotify:track:0LH7ijaQi0ybY6Oe4SCLo1',
 'spotify:track:531MBHrtGF6nAkK4zUafi9',
 'spotify:track:1otG6j1WHNvl9WgXLWkHTo',
 'spotify:track:2gAGWaK4wvt2xrFUlR4mK8',
 'spotify:track:2hPNuVVSV1tqiD2uPlfehz',
 'spotify:track:0SA0mMXWh23raZ6xzOCU2V',
 'spotify:track:26PuuuOngXVqnbnsvnOFUe',
 'spotify:track:0cciBj4236w1xByzZvn92D',
 'spotify:track:0PJIbOdMs3bd5AT8liULMQ',
 'spotify:track:5mexbTuWx9d8DPZk4sDGF4',
 'spotify:track:37lsV513gD04gFvKIPCw4N',
 'spotify:track:46CA1H0T1r7u5lDUtVveAh',
 'spotify:track:0bxmVPKnEopTyuMMkaTvUb',
 'spotify:track:5n5TzNpCrQHmsM9IlGOYqc']

In [59]:
my_playlist = sp.user_playlist_create(user=username, name="today", public=True, description="Songs for today")
a = sp.user_playlist_add_tracks(username, my_playlist['id'], uri_list)
a

{'snapshot_id': 'Myw5YzIyYjQwYzg3MDZhZWM5MTQyMjE4NTQ3YmRjYWNiMmY1MGMyYTk1'}

In [60]:
webbrowser.open(my_playlist['external_urls']['spotify'])

True

In [61]:
oura

0.64

In [62]:
tfl_min

50

In [63]:
tfl_max

100

In [64]:
fnews_score

0.19999999999999998

In [65]:
weather_mood

1